# Face Identification using Computer Vision

In [5]:
# importing libraries
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
#from keras_facenet import FaceNet
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

ImportError: Traceback (most recent call last):
  File "c:\Users\Lenovo\Projects\face_id\FaceIdEnv\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 70, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: The specified module could not be found.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [12]:

#face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') # load cascade for face detection

lando_norris_folder = r"C:\Users\caroline.kogei\Projects\Face-Identification\dataset\Lando Norris"
other_drivers_folder = r"C:\Users\caroline.kogei\Projects\Face-Identification\dataset\Other Drivers"
clean_data_Lando = r"C:\Users\caroline.kogei\Projects\Face-Identification\clean data\Lando Norris"
clean_data_Other = r"C:\Users\caroline.kogei\Projects\Face-Identification\clean data\Other Drivers"

def detect_and_crop_faces(input_folder, output_folder):
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') # load cascade for face detection
    for filename in os.listdir(input_folder):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

            for i, (x, y, w, h) in enumerate(faces):
                face = img[y:y+h, x:x+w]
                resized_face = cv2.resize(face, (160, 160))  # Resize to Facenet input size
                output_path = os.path.join(output_folder, f"{filename.split('.')[0]}_{i}.jpg")
                cv2.imwrite(output_path, resized_face)





In [13]:
# detect faces from norris and other drivers

detect_and_crop_faces(lando_norris_folder, clean_data_Lando)
detect_and_crop_faces(other_drivers_folder, clean_data_Other)

In [ ]:
# extract face embeddings with facenet

facenet_model = tf.keras.models.load_model(r'C:\Users\caroline.kogei\Projects\Face-Identification\models\20180402-114759.pb')

OSError: Unable to synchronously open file (file signature not found)

### Loading and pre-processing images

In [9]:

# defining paths to the data folders
lando_norris_folder = r"C:\Users\caroline.kogei\Projects\Face-Identification\dataset\Lando Norris"
other_drivers_folder = r"C:\Users\caroline.kogei\Projects\Face-Identification\dataset\Other Drivers"

# initialize empty list for data and labels
data = []
labels = []

# function to load and pre-process data
def load_images(folder, label):
    for filename in os.listdir(folder):
        image_path = os.path.join(folder, filename) # path to an individual image
        image = cv2.imread(image_path) # read image
        image = cv2.resize(image, (100, 100)) # resize image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to grayscale
        image = image.flatten() # flatten the image to one dimension

        # append image data and label
        data.append(image)
        labels.append(label)

# load data for both classes (Lando Norris and Other Drivers)
load_images(lando_norris_folder, 0) # label "0" for Lando Norris
load_images(other_drivers_folder, 1) # label "1" for Other Drivers



In [3]:
# saving image data as numpy arrays
array_directory = r"C:\Users\caroline.kogei\Projects\Face-Identification\clean data"

## convert data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

## save the data and labels  as .npy files
np.save(os.path.join(array_directory, 'data.npy'), data)
np.save(os.path.join(array_directory, 'labels.npy'), labels)

print("Data and labels have been saved as numpy arrays to the 'clean_data' folder.")

Data and labels have been saved as numpy arrays to the 'clean_data' folder.


### Split data into training and testing sets

In [11]:
# splitting train test data into 80/20 ratio
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

### Modelling data using Support Vector Machine (SVM)

In [12]:
# initialize and train svm model
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
# make prediction on using test data
y_pred = model.predict(X_test)
print(y_pred)

[1 1 0 1 0 1 0 0]


In [ ]:
# evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 37.50%


In [ ]:
# Print confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[2 3]
 [2 1]]
              precision    recall  f1-score   support

           0       0.50      0.40      0.44         5
           1       0.25      0.33      0.29         3

    accuracy                           0.38         8
   macro avg       0.38      0.37      0.37         8
weighted avg       0.41      0.38      0.38         8



### CNN

In [ ]:
# defining the cnn model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])


c:\Users\caroline.kogei\Projects\Face-Identification\FaceIdEnv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# compile the model
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# train model
cnn_model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5200 - loss: 20.3526 - val_accuracy: 0.4286 - val_loss: 1766.9971
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.4800 - loss: 1128.0920 - val_accuracy: 0.4286 - val_loss: 838.8706
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.4800 - loss: 532.3613 - val_accuracy: 0.4286 - val_loss: 150.7905
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.4800 - loss: 91.5544 - val_accuracy: 0.5714 - val_loss: 25.5121
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.5200 - loss: 31.6652 - val_accuracy: 0.5714 - val_loss: 12.8696
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.5200 - loss: 15.5508 - val_accuracy: 0.7143 - val_loss: 0.7047
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.8000 - loss: 0.5379 - val_accuracy: 0.4286 - val_loss: 18.3727
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.4800 - loss: 11.1870 - val_accuracy: 0

In [ ]:
# evaluate the model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3750 - loss: 1.5059
Test Accuracy: 37.50%


In [ ]:
# make prediction
def predict_image(model, image_path, img_size=(100, 100)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, img_size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img / 255.0 # normalize pixel values
    img = np.expand_dims(img, axis=-1) #channel dimension for grayscale
    img = np.expand_dims(img, axis=0) # adds batch dimension


    prediction = cnn_model.predict(img)
    return "Lando Norris" if prediction < 0.5 else "Other Drivers"

result = predict_image(model, r"dataset/Other Drivers/20.jpg")
print(f"Prediction: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Prediction: Other Drivers
